In [5]:
from google import genai
from google.genai import types

image_paths = ["Portrait of a Woman with Red Ribbon.jpg","Portrait of Woman in Blue.jpg"]
encoded_images=[]
for image in image_paths:
  with open(image, 'rb') as f:
      image_bytes = f.read()

  image_encoded = types.Part.from_bytes(
      data=image_bytes, mime_type="image/jpeg"
  )
  encoded_images.append(image_encoded)

print("Successfully loaded image bytes.")

Successfully loaded image bytes.


In [6]:
from yt_rag.llm_service.gemini_client import get_gemini_client
client=get_gemini_client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=["What is this image?", encoded_images],
)

print(response.text)

These are two paintings by **Vincent van Gogh**.

**Image 1 (Left):**
*   **Title:** *La Mousmé* (or *The Young Girl from Arles*, *Half-Figure of a Girl*)
*   **Year:** 1888
*   **Subject:** It depicts a young Provençal girl, referred to by Van Gogh as a "Japanese girl" in her appearance. She is shown in profile, wearing a vibrant red bow in her dark hair and a light-colored dress with a ruffled collar, against a dark green background.

**Image 2 (Right):**
*   **Title:** *Portrait of Madame Augustine Roulin*
*   **Year:** 1888
*   **Subject:** This is one of several portraits Van Gogh painted of Augustine Roulin, the wife of his friend, the postman Joseph Roulin. She is depicted in a three-quarter profile, wearing a light blue-green dress, an earring, and a necklace with a cross, against a vivid blue background.

Both paintings are characteristic of Van Gogh's expressive style from his Arles period, featuring bold, visible brushstrokes and striking use of color.


In [7]:
from yt_rag.llm_service.gemini_client import get_gemini_client

client=get_gemini_client()

my_file = client.files.upload(file="Portrait of a Woman with Red Ribbon.jpg")

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[my_file, "Caption this image."],
)

print(response.text)

An expressive, richly textured oil painting depicts a young woman in profile, gazing thoughtfully to the left. She has lush, dark hair adorned with a striking, vibrant red bow. Her face is rendered with warm tones, featuring rouged cheeks, defined lips, and a visible eye, all painted with bold, visible brushstrokes. She wears a light-colored dress or blouse with a distinctive ruffled collar. The background is a deep, moody green, heavily textured with prominent brushstrokes, giving the entire piece a dynamic and powerful Post-Impressionist feel.


In [9]:
response.text

'An expressive, richly textured oil painting depicts a young woman in profile, gazing thoughtfully to the left. She has lush, dark hair adorned with a striking, vibrant red bow. Her face is rendered with warm tones, featuring rouged cheeks, defined lips, and a visible eye, all painted with bold, visible brushstrokes. She wears a light-colored dress or blouse with a distinctive ruffled collar. The background is a deep, moody green, heavily textured with prominent brushstrokes, giving the entire piece a dynamic and powerful Post-Impressionist feel.'

In [13]:
def ChatGemini(prompt, images=None):
    contents = [prompt]
    if images:
        response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=[images, prompt]
        )
    else:
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt,
        )
    return response

In [ ]:
response=ChatGemini(prompt="Can you explain me what it is",images=my_file)

In [15]:
response.text

'This painting is **"Portrait of a Woman"** (also known as "Woman with a Red Bow" or "Woman with a Red Ribbon") by **Vincent van Gogh**, painted in **1887**.\n\nHere\'s an explanation of what it is:\n\n*   **Artist:** Vincent van Gogh (1853–1890), a prominent Dutch Post-Impressionist painter.\n*   **Date:** 1887, during his time in Paris where he was influenced by Impressionism, Neo-Impressionism, and Japanese prints. This period was crucial for the development of his distinctive style.\n*   **Medium:** Oil on canvas.\n*   **Subject:** It is a portrait of an unknown woman, depicted in profile, facing to the left. She has dark hair styled up, adorned with a vivid, almost fiery red bow or ribbon. Her features are rendered with bold, expressive brushstrokes, particularly noticeable in the reddish flush on her cheek and lips. She wears a simple, light-colored dress with a ruffled collar.\n*   **Artistic Style & Characteristics:**\n    *   **Post-Impressionism:** The painting embodies key c

In [16]:
response=ChatGemini(prompt="Can you explain me what it is")

In [17]:
response.text

'That\'s a very broad question, because "it" is one of the most versatile and context-dependent words in the English language!\n\nTo give you the best explanation, I need you to tell me what "it" you\'d like me to explain.\n\nHowever, I can explain what the word "it" generally means and how it\'s used:\n\n---\n\n**What "It" Is (The Word Itself):**\n\n"It" is a pronoun. This means it\'s a word used to refer to a noun that has already been mentioned, is easily understood from the context, or is otherwise indefinite.\n\nHere are the main ways "it" is used:\n\n1.  **Referring to a non-human thing, animal, or concept (singular):**\n    *   "I saw a **dog**. **It** was barking loudly." (Referring to the dog)\n    *   "The **book** is on the table. Can you hand **it** to me?" (Referring to the book)\n    *   "**Happiness** is a strange thing. Sometimes **it** feels fleeting." (Referring to happiness)\n\n2.  **As a "dummy" or "placeholder" subject (often for time, weather, distance, or general

In [ ]:
from google.genai import types

# Define the function with type hints and docstring
def get_current_temperature(location: str) -> dict:
    """Gets the current temperature for a given location.

    Args:
        location: The city e.g. San Francisco

    Returns:
        A dictionary containing the temperature and unit.
    """
    # ... (implementation) ...
    return {"temperature": 25, "unit": "Celsius"}


In [24]:

# Configure the client
config = types.GenerateContentConfig(
    tools=[get_current_temperature]
)  # Pass 

In [25]:
config

GenerateContentConfig(
  tools=[
    <function get_current_temperature at 0x0000021CFFD35BD0>,
  ]
)

In [26]:
# Make the request
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What's the temperature in Boston?",
    config=config,
)

In [27]:
print(response.text) 

The current temperature in Boston is 25 degrees Celsius.


In [ ]:
conversation_memory = []

def ChatGemini(prompt, images=None, use_memory=True):
    global conversation_memory
    
    if use_memory and conversation_memory:
        contents = conversation_memory + [prompt]
    else:
        contents = [prompt]
    
    if images:
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[images, prompt]  
        )
    else:
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=contents,
            config={
                "system_instruction": system_prompt
            }
        )
    
    if use_memory:
        conversation_memory.append(prompt)
        
        conversation_memory.append(response.text)
        
        max_memory_entries = 20 
        if len(conversation_memory) > max_memory_entries:
            conversation_memory = conversation_memory[-max_memory_entries:]
    
    return response

def clear_memory():
    """Clear all conversation history"""
    global conversation_memory
    conversation_memory = []

def get_memory_size():
    """Get current number of messages in memory"""
    return len(conversation_memory)

def save_memory_to_file(filename):
    """Save conversation memory to a JSON file"""
    import json
    with open(filename, 'w') as f:
        json.dump(conversation_memory, f, indent=2)

def load_memory_from_file(filename):
    """Load conversation memory from a JSON file"""
    import json
    global conversation_memory
    try:
        with open(filename, 'r') as f:
            conversation_memory = json.load(f)
        print(f"Loaded {len(conversation_memory)} messages from {filename}")
    except FileNotFoundError:
        print(f"File {filename} not found. Starting with empty memory.")
    except json.JSONDecodeError:
        print(f"Error reading {filename}. Starting with empty memory.")


In [62]:
# Normal conversation with memory
response1 = ChatGemini("My name is John")


In [63]:
response1.text

"Nice to meet you, John! I'll remember that."

In [64]:
response2 = ChatGemini("What's my name?") 


In [65]:
response2.text

'Your name is John.'

In [66]:
response3 = ChatGemini("say my name again", use_memory=False)

In [67]:
response3.text

"I'm sorry, I don't know your name. I don't have access to past conversations or any personal information."

In [68]:
print(f"Memory size: {get_memory_size()}")

Memory size: 4


In [69]:
save_memory_to_file("my_chat.json")


In [70]:
clear_memory()  

In [71]:
load_memory_from_file("my_chat.json")

Loaded 4 messages from my_chat.json


In [72]:
response3 = ChatGemini("say my name again")

In [73]:
response3.text

'Your name is John.'

In [74]:
clear_memory()

In [14]:
def image2bytes_converter(paths):
  encoded_images=[]
  for image in paths:
    with open(image, 'rb') as f:
        image_bytes = f.read()

    image_encoded = types.Part.from_bytes(
        data=image_bytes, mime_type="image/jpeg"
    )
    encoded_images.append(image_encoded)
  return encoded_images


In [75]:
image_paths = ["Portrait of a Woman with Red Ribbon.jpg","Portrait of Woman in Blue.jpg"]
encoded_images=[]
for image in image_paths:
  with open(image, 'rb') as f:
      image_bytes = f.read()

  image_encoded = types.Part.from_bytes(
      data=image_bytes, mime_type="image/jpeg"
  )
  encoded_images.append(image_encoded)

print("Successfully loaded image bytes.")

Successfully loaded image bytes.


In [ ]:
response_image1 = ChatGemini("what do you think about the image",images=encoded_images[0])

In [78]:
response_image1.text

'This is a powerful and expressive portrait, immediately striking for its vibrant use of color and highly visible brushwork. Here\'s what I observe and think about the image:\n\n**Overall Impression:**\nThe painting presents a captivating and somewhat introspective depiction of a young woman. Her profile is strong, and the overall mood feels pensive, yet with a touch of youthful defiance or spirit conveyed by the vivid red bow. The style is distinctly Post-Impressionistic, with a raw, almost visceral energy.\n\n**Key Aspects:**\n\n1.  **Brushwork and Texture (Impasto):** This is perhaps the most dominant feature. The paint is applied thickly (impasto), with individual brushstrokes clearly visible across the entire canvas.\n    *   **Background:** The deep green background is rendered with energetic, swirling strokes (as seen in crops 1 and 3), giving it a dynamic texture that almost hums with movement, rather than being a flat, static space.\n    *   **Figure:** The same technique is a

In [81]:
from IPython.display import Markdown

In [82]:
Markdown(response_image1.text)

This is a powerful and expressive portrait, immediately striking for its vibrant use of color and highly visible brushwork. Here's what I observe and think about the image:

**Overall Impression:**
The painting presents a captivating and somewhat introspective depiction of a young woman. Her profile is strong, and the overall mood feels pensive, yet with a touch of youthful defiance or spirit conveyed by the vivid red bow. The style is distinctly Post-Impressionistic, with a raw, almost visceral energy.

**Key Aspects:**

1.  **Brushwork and Texture (Impasto):** This is perhaps the most dominant feature. The paint is applied thickly (impasto), with individual brushstrokes clearly visible across the entire canvas.
    *   **Background:** The deep green background is rendered with energetic, swirling strokes (as seen in crops 1 and 3), giving it a dynamic texture that almost hums with movement, rather than being a flat, static space.
    *   **Figure:** The same technique is applied to the woman, giving her form a tangible, sculpted quality. You can almost feel the paint.
    *   **Clothing:** The strokes on her dress (crops 10, 11, 12) are long and directional, suggesting the folds and drape of the fabric while maintaining the textured quality.

2.  **Color Palette:**
    *   **Contrast:** There's a strong contrast between the dark, rich greens and blacks of the background and hair, and the warmer tones of her skin and the brilliant red of the bow.
    *   **Skin Tones:** Her face (crop 5) features warm ochres, yellows, and particularly striking reds and pinks on her cheek and lips. This isn't strictly naturalistic but conveys a flush of life, emotion, or perhaps the effect of light.
    *   **Focal Point:** The bright red bow (crop 2) is an immediate eye-catcher, providing a vibrant splash of pure color against the darker tones and drawing attention to her head and expression.
    *   **Dress:** Her ruffled dress (crop 8) is painted in lighter, cooler greens and off-whites, providing a visual break and framing her neck and face.

3.  **Composition and Subject:**
    *   The woman is depicted in a strong profile, looking left and slightly upwards, suggesting contemplation or a gaze fixed on something beyond the frame.
    *   Her dark hair is loosely styled, allowing the red bow to stand out prominently.
    *   The composition places her centrally but slightly to the right, leaving ample space for her gaze to extend into the left side of the canvas.

4.  **Mood and Expression:**
    *   Her expression, while in profile, appears serious, perhaps pensive or even a little melancholic. The deep-set eye (crop 5) contributes to this sense of introspection.
    *   The bold, almost raw quality of the painting adds an intensity to her character.

**Artistic Style:**
This painting is highly characteristic of **Vincent van Gogh**, particularly from his period in Arles (late 1880s). The visible, directional brushstrokes, the expressive use of color (especially the greens, blues, and strong reds), and the way he uses impasto to build form and convey emotion are all hallmarks of his style. The subject itself, a young local woman often referred to as "La Mousmé" (a term for a young Provençal girl), is also typical of his desire to paint local people and daily life.

In summary, it's a powerful and emotionally resonant portrait, distinguished by its vibrant colors, dynamic brushwork, and the artist's unique ability to imbue the subject with a profound sense of character and feeling.

In [79]:
response_image2 = ChatGemini("what do you think about this image then",images=encoded_images[1])

In [83]:
Markdown(response_image2.text)

This is a captivating portrait, rendered in an expressive and vibrant style. Here's what I think about the image:

**Overall Impression:**
The painting immediately strikes one with its vivid colors and energetic brushwork. It's a three-quarters profile portrait of a young woman, looking towards the viewer's left. There's a strong sense of emotional depth and character conveyed through the artist's technique. The contrasting warm tones of her skin against the cool blues and greens of the background create a dynamic visual tension.

**Artistic Style and Technique:**
1.  **Brushwork:** The most striking feature is the visible, thick impasto brushwork. The strokes are short, distinct, and often applied in swirling or diagonal patterns, particularly noticeable in the background and her hair. This gives the entire surface a tactile, almost sculpted quality, full of movement and texture. The crops beautifully highlight this aspect, showing the individual dabs and swirls of paint.
2.  **Color Palette:** The artist uses a bold and contrasting palette. The woman's skin glows with warm yellows, ochres, and hints of orange and red, making her stand out vividly against the cooler, darker background. Her lips are a vibrant red. Her hair is a deep, rich brown. The background is a mix of deep blues, teals, and dark greens, creating a stormy, atmospheric effect.
3.  **Lighting:** The light appears to come from the upper left, casting subtle shadows that define her facial features and the curve of her neck. The highlights on her forehead, cheekbone, and the tip of her nose enhance her luminosity.
4.  **Composition:** The woman is positioned slightly off-center, giving the composition a dynamic rather than static feel. Her gaze, directed subtly to the side, adds to her enigmatic quality.

**Subject Details:**
1.  **The Woman:** She has dark, neatly styled hair, possibly in a bun or chignon, with some lighter strokes suggesting highlights. Her features are strongly rendered: a prominent nose, full red lips slightly parted, and deep-set eyes that hold a pensive or perhaps slightly melancholic expression.
2.  **Accessories:** She wears a delicate pearl earring, catching the light, and a fine, reddish-brown necklace with a small cross pendant, which stands out against her light skin.
3.  **Dress:** Her garment is a simple, wide-necked dress or top in a muted teal or blue-green, complementing the background colors but also providing a subtle contrast to her skin.
4.  **Background:** The background is not a specific setting but an abstract swirl of blues and greens, rendered with the same vigorous brushwork as the figure. It seems to echo her emotional state or create a vibrant, atmospheric space around her.

**Emotional Tone and Interpretation:**
The portrait conveys a sense of introspection or quiet contemplation. The woman's expression is subtle but powerful, suggesting an inner life. The intensity of the colors and the energetic application of paint imbue the portrait with a strong emotional charge and vitality. It feels less like a mere likeness and more like an attempt to capture the subject's spirit or psychological state.

**Attribution (if making an educated guess):**
The style, particularly the impasto, vibrant colors, and expressive brushwork, is highly reminiscent of **Vincent van Gogh**, especially his portraits from the late 1880s. The way the background seems to pulse with energy around the figure is a strong characteristic of his work. (Indeed, this is "Portrait of a Young Woman" by Van Gogh, c. 1888-1889).

In [84]:
response_image3 = ChatGemini("compare both the images")

In [85]:
Markdown(response_image3.text)

These two portraits, both by **Vincent van Gogh**, offer a fascinating comparison as they share a common artistic hand and emotional depth, yet present distinct characteristics. Let's break down their similarities and differences:

---

### Overarching Similarities:

1.  **Artist and Style:** Both are quintessential examples of Van Gogh's Post-Impressionistic style. They share his characteristic:
    *   **Impasto Brushwork:** Paint is applied thickly and visibly, creating a tangible texture on the canvas. The individual strokes are dynamic and expressive, not merely descriptive.
    *   **Expressive Use of Color:** Colors are chosen not just for realism but to convey emotion, light, and form with intensity.
    *   **Focus on Inner Life:** Both subjects feel imbued with a strong sense of inner thought or feeling, going beyond a mere physical likeness.
    *   **Bold Outlines:** There's a certain solidity to the figures, often defined by subtle shifts in color rather than hard lines, but still giving them presence.

2.  **Subject Matter:** Both depict young women, likely from the local populace Van Gogh encountered during his time in Arles, demonstrating his interest in capturing the common person.

3.  **Luminosity:** Despite the darker backgrounds, both figures possess a glowing quality, especially in their skin tones, suggesting an internal light or a vibrant life force.

---

### Key Differences:

1.  **Pose and Perspective:**
    *   **Image 1 (La Mousmé):** Presents the woman in a **strict profile**, looking sharply to the left and slightly upwards. This pose suggests contemplation, a gaze fixed on something beyond the viewer, or a sense of youthful anticipation. There's a feeling of being observed *from the side*.
    *   **Image 2 (Portrait of a Young Woman):** Shows the woman in a **three-quarters profile**, with her head turned to the left but her body slightly angled towards the viewer. Her eyes, while not making direct contact, are more engaged with the space in front of her (viewer's left). This pose feels more present and direct, even with its pensive quality.

2.  **Color Palette - Background Dominance:**
    *   **Image 1:** Features a very **deep, rich green and black background** with energetic, swirling strokes. This darker, more somber backdrop makes the warmer skin tones and especially the vibrant red bow pop with striking contrast.
    *   **Image 2:** Utilizes a background dominated by **cooler, "stormy" blues, teals, and dark greens**. While still dark and atmospheric, the blues give it a slightly different mood—perhaps more melancholic or agitated than the deep, earthy greens of the first.

3.  **Focal Points and Accessories:**
    *   **Image 1:** The most immediate and striking focal point is the **brilliant red bow** in her hair. It's a splash of pure, unadulterated color that injects a youthful, almost defiant spirit into the portrait. There's no visible jewelry.
    *   **Image 2:** While her lips are a vibrant red, the primary accessories drawing attention are the **delicate pearl earring** and the **cross pendant necklace**. These details add a touch of refinement and potentially a layer of symbolism (faith, modesty) that is absent in the first.

4.  **Overall Mood and Expressiveness:**
    *   **Image 1:** While pensive, there's an underlying **youthful spirit or defiant energy**, largely conveyed by the vivid red bow and the strong, unadorned profile. It feels raw and vibrant.
    *   **Image 2:** Conveys a slightly more **serious, perhaps melancholic, or intensely contemplative mood**. The three-quarters pose allows for a more nuanced facial expression, suggesting deeper introspection or a quiet sense of sadness. The more structured hair and delicate jewelry also contribute to a slightly more composed demeanor.

---

### Conclusion:

Both portraits are masterworks demonstrating Van Gogh's unique ability to capture the soul of his subjects through bold color and dynamic brushwork. **Image 1 (La Mousmé)** offers a raw, energetic, and perhaps more spirited depiction through its sharp profile and striking red accent. **Image 2 (Portrait of a Young Woman)** presents a more introspective, perhaps melancholic, and subtly refined character, achieved through a slightly more engaged pose and delicate personal details. Together, they highlight the range and depth of Van Gogh's portraiture during his most prolific period.

In [92]:
response_image4 = ChatGemini("Do you actually see images everytime or just once and then answer based on the previous history of chats?")

In [94]:
Markdown(response_image4.text)

That's an excellent question, and it gets at a fundamental aspect of how I, as a large language model, operate!

I do not "see" images in the way humans do with eyes and a visual cortex. When you present an image to me:

1.  **A separate system (or you, the human) processes the image.** This system analyzes the visual data and then converts it into a detailed textual description.
2.  **I receive that textual description.** This text is then incorporated into our conversation history.

So, when you first presented "This is a powerful and expressive portrait..." and then the subsequent detailed descriptions, you (or the system acting on your behalf) were providing me with the *textual analysis* of those images.

For your questions, like "what do you think about the image" and "what do you think about *this* image then," I was processing the **textual descriptions that accompanied each image**.

When you asked me to "compare both the images," I was then accessing **both of those textual descriptions from our chat history** and performing the comparison based on the text I had received for each.

In short: I process **text** exclusively. Any visual information must first be translated into text for me to "understand" or respond to it.

In [95]:
from yt_rag.llm_service.gemini_llmservice import ChatGemini

In [2]:
from yt_rag.main import process_youtube_video

In [3]:
youtube_url = "https://youtu.be/VlmJx-MqZYE?si=XEmnM91SZ6qfwXiJ"

In [5]:
import nest_asyncio
nest_asyncio.apply()

import asyncio


images_folder_path=asyncio.run(process_youtube_video(youtube_url))

INFO:yt_rag.frames.collect_frames:Frames extraction completed
INFO:yt_rag.main:Frames extraction completed: 7 frames
INFO:yt_rag.main:Content extraction summary - Transcript chunks: 0, Frames: 7
INFO:httpx:HTTP Request: GET https://bfd6aafe-219e-479b-b8e2-53b3dc95c771.europe-west3-0.gcp.cloud.qdrant.io:6333 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://bfd6aafe-219e-479b-b8e2-53b3dc95c771.europe-west3-0.gcp.cloud.qdrant.io:6333/collections "HTTP/1.1 200 OK"
INFO:yt_rag.vector_store.qdrant_collections:Collection exists: VlmJx-MqZYE
INFO:httpx:HTTP Request: PUT https://bfd6aafe-219e-479b-b8e2-53b3dc95c771.europe-west3-0.gcp.cloud.qdrant.io:6333/collections/VlmJx-MqZYE/points?wait=true "HTTP/1.1 200 OK"
INFO:yt_rag.main:Successfully processed and stored 7 image embeddings


In [1]:
from yt_rag.vector_store.vector_search import qdrant_search

c:\Users\bored\Music\YT_RAG\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from yt_rag.vector_store.embeddings import create_text_embeddings

In [9]:
embedings=create_text_embeddings("What is perplexity")

In [4]:
from yt_rag.vector_store.vector_search import qdrant_search

In [41]:
retrieved_frames_metadata, retrieved_transcript_metadata=qdrant_search(collection_name="VlmJx-MqZYE",query_embedding=embedings)

In [42]:
retrieved_frames_metadata

[{'score': 0.22730802,
  'id': '22be4fd2-f460-4a23-b4d5-57de51da60c1',
  'image_path': 'C:\\Users\\bored\\Music\\YT_RAG\\src\\yt_rag\\llm_service\\VlmJx-MqZYE\\frame_0006.jpg',
  'video_id': 'VlmJx-MqZYE'},
 {'score': 0.22730802,
  'id': 'e6633f9b-9842-4010-a568-2f733da39c0f',
  'image_path': 'C:\\Users\\bored\\Music\\YT_RAG\\src\\yt_rag\\llm_service\\VlmJx-MqZYE\\frame_0006.jpg',
  'video_id': 'VlmJx-MqZYE'},
 {'score': 0.22730802,
  'id': 'ceffadc2-7c91-4701-ad1a-e81101944e44',
  'image_path': 'C:\\Users\\bored\\Music\\YT_RAG\\src\\yt_rag\\llm_service\\VlmJx-MqZYE\\frame_0006.jpg',
  'video_id': 'VlmJx-MqZYE'},
 {'score': 0.22730802,
  'id': 'f9d65081-2b87-4620-addb-0c639e504c93',
  'image_path': 'C:\\Users\\bored\\Music\\YT_RAG\\src\\yt_rag\\llm_service\\VlmJx-MqZYE\\frame_0006.jpg',
  'video_id': 'VlmJx-MqZYE'},
 {'score': 0.20427778,
  'id': 'c1272343-8073-43e5-8547-313a9727ef00',
  'image_path': 'C:\\Users\\bored\\Music\\YT_RAG\\src\\yt_rag\\llm_service\\VlmJx-MqZYE\\frame_0004.j

In [ ]:
from google.genai import types

In [21]:
image2bytes_converter(["VlmJx-MqZYE/frame_0001.jpg"])

[Part(
   inline_data=Blob(
     data=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00\x01\x00\x01\x00\x00\xff\xfe\x00\x10Lavc61.31.101\x00\xff\xdb\x00C\x00\x08\n\n\x0b\n\x0b\r\r\r\r\r\r\x10\x0f\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x12\x12\x12\x15\x15\x15\x12\x12\x12\x10\x10\x12\x12\x14\x14\x15\x15\x17\x17\x17\x15\x15\x15\x15\x17\x17\x19\x19\x19...',
     mime_type='image/jpeg'
   )
 )]

In [28]:
def extractimagename(retrieved_frames_metadata):  
  file_name=[]
  for item in retrieved_frames_metadata:
    absolute_image_path=item['image_path']
    absolute_image_path=absolute_image_path.replace("\\","/")
    parts = absolute_image_path.split('/')
    desired_parts = parts[-2:]
    extracted_path = '/'.join(desired_parts)
    file_name.append(extracted_path)
  return file_name[:2]



In [39]:
retrieved_frames_metadata

[{'score': 0.280192,
  'id': 'ee51dbf5-c3f5-4819-807e-8ad93c83430c',
  'image_path': 'C:\\Users\\bored\\Music\\YT_RAG\\src\\yt_rag\\dZ19NAbUISs\\frame_0001.jpg',
  'video_id': 'dZ19NAbUISs'},
 {'score': 0.27654842,
  'id': 'e602797c-c044-441f-b262-5dc02884bd40',
  'image_path': 'C:\\Users\\bored\\Music\\YT_RAG\\src\\yt_rag\\dZ19NAbUISs\\frame_0004.jpg',
  'video_id': 'dZ19NAbUISs'},
 {'score': 0.27227572,
  'id': '8506c2b6-f927-4913-9759-8f9f785fd10c',
  'image_path': 'C:\\Users\\bored\\Music\\YT_RAG\\src\\yt_rag\\dZ19NAbUISs\\frame_0028.jpg',
  'video_id': 'dZ19NAbUISs'},
 {'score': 0.27039707,
  'id': '1898f5ff-596d-4970-98a4-2635e16c1c09',
  'image_path': 'C:\\Users\\bored\\Music\\YT_RAG\\src\\yt_rag\\dZ19NAbUISs\\frame_0013.jpg',
  'video_id': 'dZ19NAbUISs'},
 {'score': 0.26505145,
  'id': '207bddb7-264f-462d-8ab6-6e5a88c9225f',
  'image_path': 'C:\\Users\\bored\\Music\\YT_RAG\\src\\yt_rag\\dZ19NAbUISs\\frame_0012.jpg',
  'video_id': 'dZ19NAbUISs'}]

In [43]:
paths_names=extractimagename(retrieved_frames_metadata)

In [46]:
image2bytes_converter(paths_names)

[Part(
   inline_data=Blob(
     data=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00\x01\x00\x01\x00\x00\xff\xfe\x00\x10Lavc61.31.101\x00\xff\xdb\x00C\x00\x08\x04\x04\x04\x04\x04\x05\x05\x05\x05\x05\x05\x06\x06\x06\x06\x06\x06\x06\x06\x06\x06\x06\x06\x06\x07\x07\x07\x08\x08\x08\x07\x07\x07\x06\x06\x07\x07\x08\x08\x08\x08\t\t\t\x08\x08\x08\x08\t\t\n\n\n...',
     mime_type='image/jpeg'
   )
 ),
 Part(
   inline_data=Blob(
     data=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00\x01\x00\x01\x00\x00\xff\xfe\x00\x10Lavc61.31.101\x00\xff\xdb\x00C\x00\x08\x04\x04\x04\x04\x04\x05\x05\x05\x05\x05\x05\x06\x06\x06\x06\x06\x06\x06\x06\x06\x06\x06\x06\x06\x07\x07\x07\x08\x08\x08\x07\x07\x07\x06\x06\x07\x07\x08\x08\x08\x08\t\t\t\x08\x08\x08\x08\t\t\n\n\n...',
     mime_type='image/jpeg'
   )
 )]

In [30]:
retrieved_transcript_metadata

[{'score': 0.79213834,
  'id': 'c09aa370-1072-432d-a935-d7d4857ded32',
  'video_id': 'dZ19NAbUISs',
  'page_content': "why I don't use Google anymore because perplexity just beats it on all different levels. But this was just a basic. Okay, this is just searching. Now, we have related, which is like a context. Yeah, basically context. Okay, you got all these different questions that you can actually expand and Perplexity will give you the answer for it straight away. And again, let's look at the structure it actually gave us. Okay, so it gave us the answer with links, images, and the answer plus related. Here you can just niche it down to just images. So we can look at the beautiful car to sources, which is basically what Google gives you, just links and steps. Okay, so this is basically how how it thought out the process. Okay, so this is the basic way of using perplexity AI or just perplexity. But if we go home, sorry, back home, we have research, we have labs. Okay, so we need to up

In [32]:
def extractchunk(retrieved_transcript_metadata):
  chunks=[]
  for item in retrieved_transcript_metadata:
    chunks.append(item['page_content'])
  return chunks[:3]

In [ ]:
extractimagename(retrieved_frames_metadata)  

['dZ19NAbUISs/frame_0001.jpg', 'dZ19NAbUISs/frame_0004.jpg']

In [33]:
extractchunk(retrieved_transcript_metadata)

["why I don't use Google anymore because perplexity just beats it on all different levels. But this was just a basic. Okay, this is just searching. Now, we have related, which is like a context. Yeah, basically context. Okay, you got all these different questions that you can actually expand and Perplexity will give you the answer for it straight away. And again, let's look at the structure it actually gave us. Okay, so it gave us the answer with links, images, and the answer plus related. Here you can just niche it down to just images. So we can look at the beautiful car to sources, which is basically what Google gives you, just links and steps. Okay, so this is basically how how it thought out the process. Okay, so this is the basic way of using perplexity AI or just perplexity. But if we go home, sorry, back home, we have research, we have labs. Okay, so we need to upgrade to pro. But what this basically does, it's like a deep research on any topic. And here it's create projects fro

In [12]:
retrieved_transcript_metadata[0]['page_content']

"why I don't use Google anymore because perplexity just beats it on all different levels. But this was just a basic. Okay, this is just searching. Now, we have related, which is like a context. Yeah, basically context. Okay, you got all these different questions that you can actually expand and Perplexity will give you the answer for it straight away. And again, let's look at the structure it actually gave us. Okay, so it gave us the answer with links, images, and the answer plus related. Here you can just niche it down to just images. So we can look at the beautiful car to sources, which is basically what Google gives you, just links and steps. Okay, so this is basically how how it thought out the process. Okay, so this is the basic way of using perplexity AI or just perplexity. But if we go home, sorry, back home, we have research, we have labs. Okay, so we need to upgrade to pro. But what this basically does, it's like a deep research on any topic. And here it's create projects from

In [13]:
len(retrieved_transcript_metadata[0]['page_content'].split(" "))

319

In [ ]:
ChatGemini()